In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained(
    "roberta-base",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
class ComplaintDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=64):
        self.texts = dataframe["Complaint/ Opinion"].astype(str).tolist()
        self.labels = dataframe["Label"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoded = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

In [4]:
class BiGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(BiGRUModel, self).__init__()
        self.bigru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )

    def forward(self, x):
        output, hidden = self.bigru(x)
        return output, hidden

In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_size * 2, 1)

    def forward(self, gru_output):
        scores = self.attention(gru_output)             
        attn_weights = F.softmax(scores, dim=1)          
        context_vector = torch.sum(attn_weights * gru_output, dim=1)  
        return context_vector, attn_weights

In [6]:
class CentralBrain(nn.Module):
    def __init__(self, roberta_hidden_size=768, gru_hidden_size=128, num_classes=2):
        super(CentralBrain, self).__init__()
        self.central_fc = nn.Linear(roberta_hidden_size + gru_hidden_size * 2, 128)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, cls_embedding, attn_output):
        combined = torch.cat((cls_embedding, attn_output), dim=1)
        central_output = torch.relu(self.central_fc(combined))
        logits = self.classifier(central_output)
        return logits

In [7]:
class FullModel(nn.Module):
    def __init__(self, roberta, gru_hidden_size=128, num_classes=2):
        super(FullModel, self).__init__()
        self.roberta = roberta
        self.bigru = BiGRUModel(input_size=768, hidden_size=gru_hidden_size)
        self.attention = Attention(gru_hidden_size)
        self.central_brain = CentralBrain(768, gru_hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state    # [batch, seq_len, 768]
        cls_embedding = outputs.pooler_output          # [batch, 768]

        gru_output, _ = self.bigru(sequence_output)    # [batch, seq_len, hidden*2]
        attn_output, attn_weights = self.attention(gru_output)  # [batch, hidden*2]

        logits = self.central_brain(cls_embedding, attn_output) # [batch, num_classes]
        return logits

In [8]:
def train_model(model, train_loader, epochs=3, lr=2e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

from sklearn.model_selection import train_test_split
df = pd.read_excel('C:\\Users\\BIT\\Downloads\\Complain.xlsx')
df.rename(columns={'Complaint Label':'Label'}, inplace=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Label'])

train_dataset = ComplaintDataset(train_df, tokenizer, max_len=64)
test_dataset = ComplaintDataset(test_df, tokenizer, max_len=64)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model = FullModel(roberta, gru_hidden_size=128, num_classes=2).to(device)


for param in model.roberta.parameters():
    param.requires_grad = False

print("\n🚀 Starting training...")
train_model(model, train_loader, epochs=5, lr=2e-5)

In [12]:
from sklearn.model_selection import train_test_split
df = pd.read_excel('C:\\Users\\BIT\\Downloads\\Complain.xlsx')
df.rename(columns={'Complaint Label':'Label'}, inplace=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Label'])

In [10]:
import torch
from transformers import RobertaTokenizer

def predict_complaint_from_text(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = RobertaTokenizer.from_pretrained("saved_model-Copy1")
    loaded_model = FullModel(roberta, gru_hidden_size=128, num_classes=2).to(device)
    loaded_model.load_state_dict(torch.load("saved_model/full_model.pth", map_location=device))
    loaded_model.eval()

    encoded = tokenizer(
        text,
        max_length=64,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    with torch.no_grad():
        logits = loaded_model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    #print(f"Input: {text}")
    #print(f"Prediction class: {predicted_class} ({'complaint' if predicted_class==1 else 'non complaint'})")
    #print(f"Probabilities: {probs.cpu().numpy()}")
    return predicted_class

text = "@LICIndiaForever @Paytmcare @Paytm As part of my LIC premium renewal my balance deducted but payment status shows fail. I have used LIC website where my payment gateways is Paytm. Transaction ID- 31454836 Kindly address."
predict_complaint_from_text(text)

    

1

In [13]:
test_output=test_df['Label']
from sklearn.metrics import r2_score

In [14]:
test_df

,Domain,Complaint/ Opinion,Label,Complaint_Cause,Severity level,Sentiment,Emotion
2151,Transaction Failure,@MyIndianBank @DFS_India Kindly recheck the li...,1,i could not set the transaction date,No explicit reproach,negative,sadness
3917,Crypto,@Elliot__Elliot @MrHodl @jack @morcosa @cz_bin...,0,no cause,Non-Complaint,neutral,other
1145,Customer Service,@AmazonHelp This was my order no 404-2135543-8...,1,no response,No explicit reproach,negative,sadness
4989,Debit card,@drscottbelshaw @UNTcyberlab @CBS11Andrea @Kat...,0,no cause,Non-Complaint,negative,fear
5774,Stocks and investment,Good Morning. Dont know who wants to hear this...,0,no cause,Non-Complaint,negative,fear
...,...,...,...,...,...,...,...
3149,Transaction Fail,@ICICIBank_Care thank you for resolving issue ...,0,no cause,Non-Complaint,positive,happiness
6103,Financial Policies,"@MandarKagade Well, the amendments made to RBI...",0,no cause,Non-Complaint,negative,fear
674,Customer Service,@TheOfficialSBI @SBICard_Connect \nAp no.24358...,1,Very poor response from verification guy,Blame,negative,sadness
5603,Stocks and investment,"@invest_mutual 1. Last year ,they had the best...",0,no cause,Non-Complaint,positive,happiness


In [15]:
test_output = test_df['Label'].values       
predict = []

for i in test_df['Complaint/ Opinion']:
    j = predict_complaint_from_text(i)       
    predict.append(j)

predict = np.array(predict)

print("R2 Score:", r2_score(test_output, predict))  

NameError: name 'np' is not defined

In [17]:
import numpy as np
predict = np.array(predict)

print("R2 Score:", r2_score(test_output, predict))  

R2 Score: 0.7038186520614635
